# Exercices

## **Préliminaires**: Clone de votre repo et imports

In [4]:
! git clone https://github.com/remi421/exam_2025.git
! cp exam_2025/utils/utils_exercices.py .

import copy
import numpy as np
import torch

Cloning into 'exam_2025'...
remote: Enumerating objects: 59, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 59 (delta 21), reused 20 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (59/59), 1.41 MiB | 5.40 MiB/s, done.
Resolving deltas: 100% (21/21), done.


**Clef personnelle pour la partie théorique**

Dans la cellule suivante, choisir un entier entre 100 et 1000 (il doit être personnel). Cet entier servira de graine au générateur de nombres aléatoire a conserver pour tous les exercices.



In [5]:
mySeed = 200

\

---

\

\

**Exercice 1** *Une relation linéaire*

La fonction *generate_dataset* fournit deux jeux de données (entraînement et test). Pour chaque jeu de données, la clef 'inputs' donne accès à un tableau numpy (numpy array) de prédicteurs empilés horizontalement : chaque ligne $i$ contient trois prédicteurs $x_i$, $y_i$ et $z_i$. La clef 'targets' renvoie le vecteur des cibles $t_i$. \

Les cibles sont liées aux prédicteurs par le modèle:
$$ t = \theta_0 + \theta_1 x + \theta_2 y + \theta_3 z + \epsilon$$ où $\epsilon \sim \mathcal{N}(0,\eta)$


In [6]:
from utils_exercices import generate_dataset, Dataset1
train_set, test_set = generate_dataset(mySeed)

**Q1** Par quelle méthode simple peut-on estimer les coefficients $\theta_k$ ? La mettre en oeuvre avec la librairie python de votre choix.

On peut estimer les coefficients θk en faisant une régression multiple. Nous avons 3 prédicteurs (x,y,z) et une variable cible (t). La régression linéaire multiple trouvera les coefficients θk qui minimisent l'erreur entre les valeurs prédites par le modèle et les valeurs réelles de t.

In [7]:
import numpy as np
from sklearn.linear_model import LinearRegression

# Créer un modèle de régression linéaire
model = LinearRegression()

# Ajuster le modèle aux données d'entraînement
X = train_set['inputs']
y = train_set['targets']
model.fit(X, y)

# Afficher les coefficients estimés
print(model.coef_)
print(model.intercept_)

[1.95156862 1.94842221 3.59966699]
10.078764034363882


**Q2** Dans les cellules suivantes, on se propose d'estimer les $\theta_k$ grâce à un réseau de neurones entraîné par SGD. Quelle architecture s'y prête ? Justifier en termes d'expressivité et de performances en généralisation puis la coder dans la cellule suivante.

L'architecture qui se prête le mieux à ce problème est un réseau de neurones avec une seule "hidden layer" (avec une fonction d'activation linéaire).


En effet, on cherche à modéliser une relation linéaire entre les prédicteurs et la variable cible.

En termes de performances en généralisation, Un réseau de neurones simple avec un nombre limité de paramètres est moins susceptible de surajuster les données d'entraînement, ce qui signifie qu'il sera plus performant sur des données non vues.

In [9]:
import torch.nn as nn
import torch.optim as optim
# Dataset et dataloader :
dataset = Dataset1(train_set['inputs'], train_set['targets'])
dataloader = torch.utils.data.DataLoader(dataset, batch_size=100, shuffle=True)

# A coder :
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(3, 1)  # 3 entrées (x, y, z) et 1 sortie (t)

    def forward(self, x):
        x = self.fc1(x)
        return x

**Q3** Entraîner cette architecture à la tâche de régression définie par les entrées et sorties du jeu d'entraînement (compléter la cellule ci-dessous).

In [11]:
# Initialize model, loss, and optimizer
mySimpleNet = SimpleNet()
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(mySimpleNet.parameters(), lr=0.01)

# Training loop
num_epochs = 500
for epoch in range(num_epochs):
    for batch_inputs, batch_targets in dataloader:


        optimizer.zero_grad()
        outputs = mySimpleNet(batch_inputs)
        loss = criterion(outputs, batch_targets.unsqueeze(1))
        loss.backward()
        optimizer.step()


**Q4** Où sont alors stockées les estimations des  $\theta_k$ ? Les extraire du réseau *mySimpleNet* dans la cellule suivante.

Les estimations des θk sont situées dans les attributs de la couche linéaire fc1 de notre modèle.



In [12]:
# Extraire les poids (θ1, θ2, θ3)
weights = mySimpleNet.fc1.weight.data.numpy()

# Extraire le biais (θ0)
bias = mySimpleNet.fc1.bias.data.numpy()

# Afficher les estimations de θk
print("Poids (θ1, θ2, θ3):", weights)
print("Biais (θ0):", bias)

Poids (θ1, θ2, θ3): [[1.9521308 1.9476329 3.5980055]]
Biais (θ0): [10.077012]


**Q5** Tester ces estimations sur le jeu de test et comparer avec celles de la question 1. Commentez.

In [19]:
from sklearn.metrics import mean_squared_error
# Convertir les données de test en tenseurs PyTorch
X_test = torch.from_numpy(test_set['inputs']).float()
y_test = torch.from_numpy(test_set['targets']).float()

# Prédire les valeurs de la variable cible sur le jeu de test
with torch.no_grad():  # Désactiver le calcul des gradients pour l'inférence
    predictions = mySimpleNet(X_test)

# Calculer l'erreur quadratique moyenne (MSE)
mse = torch.mean((predictions.squeeze() - y_test)**2)

# Afficher la MSE
print("MSE du réseau de neurones :", mse.item())

# MSE provenant du modèle de la q1
predictions_lr = model.predict(test_set['inputs']) # Predictions du modèle de régression linéaire sur les données de test
mse_lr = mean_squared_error(test_set['targets'], predictions_lr)
print("MSE de la régression linéaire :", mse_lr)


MSE du réseau de neurones : 4.0100483894348145
MSE de la régression linéaire : 4.0092609424969


Les 2 méthodes obtiennent des performances similaires sur le jeu de test.Ce n'est pas étonnant car la relation entre les prédicteurs et la variable cible est linéaire, et les deux méthodes sont capables de modéliser correctement ce type de relation.

\

---

\

**Exercice 2** *Champ réceptif et prédiction causale*

Le réseau défini dans la cellule suivante est utilisé pour faire le lien entre les valeurs $(x_{t' \leq t})$ d'une série temporelle d'entrée et la valeur présente $y_t$ d'une série temporelle cible.

In [20]:
import torch.nn as nn
import torch.nn.functional as F
from utils_exercices import Outconv, Up_causal, Down_causal

class Double_conv_causal(nn.Module):
    '''(conv => BN => ReLU) * 2, with causal convolutions that preserve input size'''
    def __init__(self, in_ch, out_ch, kernel_size=3, dilation=1):
        super(Double_conv_causal, self).__init__()
        self.kernel_size = kernel_size
        self.dilation = dilation
        self.conv1 = nn.Conv1d(in_ch, out_ch, kernel_size=kernel_size, padding=0, dilation=dilation)
        self.bn1 = nn.BatchNorm1d(out_ch)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv1d(out_ch, out_ch, kernel_size=kernel_size, padding=0, dilation=dilation)
        self.bn2 = nn.BatchNorm1d(out_ch)

    def forward(self, x):
        x = F.pad(x, ((self.kernel_size - 1) * self.dilation, 0))
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = F.pad(x, ((self.kernel_size - 1) * self.dilation, 0))
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        return x


class causalFCN(nn.Module):
    def __init__(self, dilation=1):
        super(causalFCN, self).__init__()
        size = 64
        n_channels = 1
        n_classes = 1
        self.inc = Double_conv_causal(n_channels, size)
        self.down1 = Down_causal(size, 2*size)
        self.down2 = Down_causal(2*size, 4*size)
        self.down3 = Down_causal(4*size, 8*size, pooling_kernel_size=5, pooling_stride=5)
        self.down4 = Down_causal(8*size, 4*size, pooling=False, dilation=2)
        self.up2 = Up_causal(4*size, 2*size, kernel_size=5, stride=5)
        self.up3 = Up_causal(2*size, size)
        self.up4 = Up_causal(size, size)
        self.outc = Outconv(size, n_classes)
        self.n_classes = n_classes

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up2(x5, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        x = self.outc(x)
        return x

# Exemple d'utilisation
model = causalFCN()
# Série temporelle d'entrée (x_t):
input_tensor1 = torch.rand(1, 1, 10000)
# Série temporelle en sortie f(x_t):
output = model(input_tensor1)
print(output.shape)

torch.Size([1, 1, 10000])


**Q1** De quel type de réseau de neurones s'agit-il ? Combien de paramètres la couche self.Down1 compte-t-elle (à faire à la main) ?
Combien de paramètres le réseau entier compte-t-il (avec un peu de code) ?

Le réseau causalFCN est un réseau de neurones convolutif complètement causal. Il a une architecture similaire à celle d'un U-Net.

In [ ]:
# Nb de paramètres dans self.Down1: (calcul "à la main")



# Nb de paramètres au total:

**Q2** Par quels mécanismes la taille du vecteur d'entrée est-elle réduite ? Comment est-elle restituée dans la deuxième partie du réseau ?

**Q3** Par quels mécanismes le champ réceptif est-il augmenté ? Préciser par un calcul la taille du champ réceptif en sortie de *self.inc*.

**Q4** Par un bout de code, déterminer empiriquement la taille du champ réceptif associé à la composante $y_{5000}$ du vecteur de sortie. (Indice: considérer les sorties associées à deux inputs qui ne diffèrent que par une composante...)

**Q5** $y_{5000}$ dépend-elle des composantes $x_{t, \space t > 5000}$ ? Justifier de manière empirique puis préciser la partie du code de Double_conv_causal qui garantit cette propriété de "causalité" en justifiant.  



\

---

\

\

Exercice 3: "Ranknet loss"

Un [article récent](https://https://arxiv.org/abs/2403.14144) revient sur les progrès en matière de learning to rank. En voilà un extrait :


<img src="https://raw.githubusercontent.com/nanopiero/exam_2025/refs/heads/main/utils/png_exercice3.PNG?token=GHSAT0AAAAAAC427DACOPGNDNN6UDOLVLLAZ4BB2JQ" alt="extrait d'un article" width="800">

**Q1** Qu'est-ce que les auteurs appellent "positive samples" et "negative samples" ? Donner un exemple.

**Q2** Dans l'expression de $\mathcal{L}_{RankNet}$, d'où proviennent les $z_i$ ? Que représentent-ils ?  

**Q3** Pourquoi cette expression conduit-elle à ce que, après apprentissage, "the estimated
value of positive samples is greater than that of negative samples
for each pair of positive/negative samples" ?

**Q4** Dans le cadre d'une approche par deep learning, quels termes utilise-t-on pour qualifier les réseaux de neurones exploités et la modalité suivant laquelle ils sont entraînés ?